In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [3]:
import pipit as pp
import pandas as pd
import numpy as np
import polars as pl

2024-04-25 00:08:20,159	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
# avoid the reader for now
trace = pp.Trace.from_otf2("/home/rakrish/Downloads/tortuga-otf2-16/")
trace.events.drop("Thread", axis=1, inplace=True)
trace.events

,Timestamp (ns),Event Type,Name,Process,Attributes
0,0.000000e+00,Instant,ProgramBegin,6,"{'attributes': {'Attribute 2': 35468}, 'progra..."
1,4.868378e+01,Instant,ProgramBegin,7,"{'attributes': {'Attribute 2': 35471}, 'progra..."
2,2.539193e+02,Instant,ProgramBegin,5,"{'attributes': {'Attribute 2': 35465}, 'progra..."
3,3.035577e+02,Instant,ProgramBegin,4,"{'attributes': {'Attribute 2': 35462}, 'progra..."
4,4.946367e+04,Instant,ProgramBegin,3,"{'attributes': {'Attribute 2': 35460}, 'progra..."
...,...,...,...,...,...
95611,1.748975e+09,Instant,MeasurementOnOff,4,{'measurement_mode': 'MeasurementMode.OFF'}
95612,1.749001e+09,Leave,run,12,None
95613,1.749008e+09,Instant,MeasurementOnOff,12,{'measurement_mode': 'MeasurementMode.OFF'}
95614,1.749750e+09,Leave,run,0,None


In [5]:
pp.get_option("backend")

'ray'

In [6]:
dict_trace = pp.DictTrace(trace.events)

2024-04-25 00:08:26,431	INFO worker.py:1749 -- Started a local Ray instance.


In [8]:
dict_trace._match_events()

In [9]:
dict_trace.dfs[2].query("`Event Type`.isin(['Enter', 'Leave'])")

,Timestamp (ns),Event Type,Name,Process,Attributes,_matching_event
1,8.986357e+04,Enter,MPI_Init,2,{'region': 'Region 36'},2
2,5.509726e+08,Leave,MPI_Init,2,None,1
5,8.264942e+08,Enter,run,2,{'region': 'Region 89'},5974
7,8.265009e+08,Enter,MPI_Bcast,2,{'region': 'Region 10'},9
9,8.272976e+08,Leave,MPI_Bcast,2,None,7
...,...,...,...,...,...,...
5968,1.706474e+09,Leave,MPI_Barrier,2,None,5966
5970,1.748596e+09,Enter,MPI_Allreduce,2,{'region': 'Region 5'},5972
5972,1.748624e+09,Leave,MPI_Allreduce,2,None,5970
5973,1.748629e+09,Leave,writeSingleField,2,None,5944


In [18]:
dict_trace.dfs[0]

,Timestamp (ns),Event Type,Name,Process,Attributes,_matching_event
0,4.959588e+04,Instant,ProgramBegin,0,"{'attributes': {'Attribute 2': 35455}, 'progra...",NaN
1,9.251779e+04,Enter,MPI_Init,0,{'region': 'Region 36'},2.0
2,5.509727e+08,Leave,MPI_Init,0,None,1.0
3,5.509812e+08,Instant,MeasurementOnOff,0,{'measurement_mode': 'MeasurementMode.OFF'},NaN
4,8.267803e+08,Instant,MeasurementOnOff,0,{'measurement_mode': 'MeasurementMode.ON'},NaN
...,...,...,...,...,...,...
5971,1.748623e+09,Instant,MpiCollectiveEnd,0,"{'collective_op': 'CollectiveOp.ALLREDUCE', 'c...",NaN
5972,1.748624e+09,Leave,MPI_Allreduce,0,None,5970.0
5973,1.748661e+09,Leave,writeSingleField,0,None,5944.0
5974,1.749750e+09,Leave,run,0,None,5.0


In [7]:
dict_trace._match_caller_callee()
dict_trace.dfs[2]
# dict_trace.dfs[2].query("(`Event Type`.isin(['Enter', 'Leave'])) & (`_parent`.notnull())")

,Timestamp (ns),Event Type,Name,Process,Attributes,_matching_event,_depth,_parent
0,4.992140e+04,Instant,ProgramBegin,2,"{'attributes': {'Attribute 2': 35458}, 'progra...",<NA>,<NA>,<NA>
1,8.986357e+04,Enter,MPI_Init,2,{'region': 'Region 36'},2,0,<NA>
2,5.509726e+08,Leave,MPI_Init,2,None,1,<NA>,<NA>
3,5.509812e+08,Instant,MeasurementOnOff,2,{'measurement_mode': 'MeasurementMode.OFF'},<NA>,<NA>,<NA>
4,8.264770e+08,Instant,MeasurementOnOff,2,{'measurement_mode': 'MeasurementMode.ON'},<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...
5971,1.748623e+09,Instant,MpiCollectiveEnd,2,"{'collective_op': 'CollectiveOp.ALLREDUCE', 'c...",<NA>,<NA>,<NA>
5972,1.748624e+09,Leave,MPI_Allreduce,2,None,5970,<NA>,<NA>
5973,1.748629e+09,Leave,writeSingleField,2,None,5944,<NA>,<NA>
5974,1.748798e+09,Leave,run,2,None,5,<NA>,<NA>


In [34]:
dict_trace.dfs[2].dtypes


Timestamp (ns)      float64
Event Type         category
Name               category
Process            category
Attributes           object
_matching_event     float64
_depth              float64
_parent             float64
dtype: object

In [9]:
dict_trace.calc_inc_metrics()
dict_trace.events.filter((pl.col("Event Type").is_in(["Enter", "Leave"])) & (pl.col("_parent").is_not_null()))

index,Timestamp (ns),Event Type,Name,Thread,Process,Attributes,_matching_event,_depth,_parent,_matching_timestamp,time.inc
u32,f64,cat,cat,i64,i64,struct[15],u32,u32,u32,f64,f64
68,8.2534e8,"""Enter""","""MPI_Bcast""",0,4,"{{null},null,null,null,null,null,null,null,null,""Region 10"",null,null,null,null,null}",155,1,65,8.2730e8,1.9573e6
71,8.2535e8,"""Enter""","""MPI_Bcast""",0,6,"{{null},null,null,null,null,null,null,null,null,""Region 10"",null,null,null,null,null}",150,1,70,8.2730e8,1.9449e6
75,8.2537e8,"""Enter""","""MPI_Bcast""",0,12,"{{null},null,null,null,null,null,null,null,null,""Region 10"",null,null,null,null,null}",183,1,73,8.2730e8,1.9333e6
80,8.2549e8,"""Enter""","""MPI_Bcast""",0,7,"{{null},null,null,null,null,null,null,null,null,""Region 10"",null,null,null,null,null}",151,1,77,8.2730e8,1.8073e6
85,8.2551e8,"""Enter""","""MPI_Bcast""",0,14,"{{null},null,null,null,null,null,null,null,null,""Region 10"",null,null,null,null,null}",181,1,82,8.2730e8,1.7944e6
…,…,…,…,…,…,…,…,…,…,…,…
95526,1.7486e9,"""Enter""","""MPI_Allreduce""",0,12,"{{null},null,null,null,null,null,null,null,null,""Region 5"",null,null,null,null,null}",95549,2,95107,1.7486e9,26609.026294
95528,1.7486e9,"""Enter""","""MPI_Allreduce""",0,1,"{{null},null,null,null,null,null,null,null,null,""Region 5"",null,null,null,null,null}",95564,2,95115,1.7486e9,26899.219801
95531,1.7486e9,"""Enter""","""MPI_Allreduce""",0,11,"{{null},null,null,null,null,null,null,null,null,""Region 5"",null,null,null,null,null}",95562,2,95111,1.7486e9,26518.340823


In [10]:
dict_trace.calc_exc_metrics()
dict_trace.events.filter(pl.col("time.exc") < pl.col("time.inc"))

index,Timestamp (ns),Event Type,Name,Thread,Process,Attributes,_matching_event,_depth,_parent,_matching_timestamp,time.inc,time.exc
u32,f64,cat,cat,i64,i64,struct[15],u32,u32,u32,f64,f64,f64
65,8.2533e8,"""Enter""","""run""",0,4,"{{null},null,null,null,null,null,null,null,null,""Region 89"",null,null,null,null,null}",95610,0,null,1.7490e9,9.2364e8,1.5056e7
70,8.2535e8,"""Enter""","""run""",0,6,"{{null},null,null,null,null,null,null,null,null,""Region 89"",null,null,null,null,null}",95586,0,null,1.7488e9,9.2346e8,1.4538e7
73,8.2536e8,"""Enter""","""run""",0,12,"{{null},null,null,null,null,null,null,null,null,""Region 89"",null,null,null,null,null}",95612,0,null,1.7490e9,9.2364e8,1.5869e7
77,8.2549e8,"""Enter""","""run""",0,7,"{{null},null,null,null,null,null,null,null,null,""Region 89"",null,null,null,null,null}",95588,0,null,1.7488e9,9.2333e8,1.4895e7
82,8.2550e8,"""Enter""","""run""",0,14,"{{null},null,null,null,null,null,null,null,null,""Region 89"",null,null,null,null,null}",95593,0,null,1.7488e9,9.2332e8,1.5034e7
…,…,…,…,…,…,…,…,…,…,…,…,…
95114,1.4148e9,"""Enter""","""writeSingleFie…",0,3,"{{null},null,null,null,null,null,null,null,null,""Region 99"",null,null,null,null,null}",95579,1,109,1.7486e9,3.3382e8,3.3058e8
95115,1.4148e9,"""Enter""","""writeSingleFie…",0,1,"{{null},null,null,null,null,null,null,null,null,""Region 99"",null,null,null,null,null}",95580,1,117,1.7486e9,3.3382e8,3.2965e8
95116,1.4148e9,"""Enter""","""writeSingleFie…",0,10,"{{null},null,null,null,null,null,null,null,null,""Region 99"",null,null,null,null,null}",95577,1,98,1.7486e9,3.3382e8,3.3083e8


In [11]:
dict_trace.comm_matrix()

array([[  0.,   0.,   0.,   0.,  73.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,  73.,   0., 146., 146.],
       [  0.,   0., 146., 146.,   0.,  73.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,  73.,   0.,   0.],
       [  0., 146.,   0.,   0., 146.,   0.,  73.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,  73.],
       [  0., 146.,   0.,   0., 146.,   0.,   0.,  73.,   0.,   0.,   0.,
          0.,   0.,   0.,  73.,   0.],
       [ 73.,   0., 146., 146.,   0.,   0.,   0.,   0.,  73.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,  73.,   0.,   0.,   0.,   0., 146., 146.,   0.,  73.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,  73.,   0.,   0., 146.,   0.,   0., 146.,   0.,  73.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,  73.,   0., 146.,   0.,   0., 146.,   0.,   0.,
         73.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,  73.,   0., 146., 146.,   0.,   0.,   0.,
          0.,  73.,   

In [16]:
dict_trace.message_histogram()

category,count
cat,u32
"""(-inf, 16383.0…",0
"""(16383.0, 2293…",6912
"""(22936.7, 2949…",0
"""(29490.4, 3604…",0
"""(36044.1, 4259…",0
…,…
"""(114688.5, 121…",0
"""(121242.2, 127…",0
"""(127795.9, 134…",0


In [13]:
# dict_trace.events.with_columns(
#     pl.col("Attributes").struct.field("msg_length")
# ).cast({"Timestamp (ns)": pl.Int64}).set_sorted("Timestamp (ns)").group_by_dynamic("Timestamp (ns)", every="87487933i").agg(pl.col("msg_length").sum())

Timestamp (ns),msg_length
i64,i64
0,0
262463799,0
524927598,0
787391397,23592960
874879330,118112256
…,…
1224831062,109658112
1312318995,0
1399806928,0


In [ ]:
# dict_trace.events.with_columns([
#     pl.col("Attributes").struct.field("msg_length"),
#     pl.col("Timestamp (ns").cast(pl.UInt64)
# ]).set_sorted("Timestamp (ns)").group_by_dynamic("Timestamp (ns)", every="1us").agg(pl.col("msg_length").sum())